## De-Duping the CSV

In order to make sure that the data in the original Google sheet was correct, we kept more columns than we needed. In this notebook, we are going to examine the columns in the "one to rule them all" notebook and decide which columns can be deleted. The original merged notebook we will be unaffected as we will be working with a duplicate file.

In [2]:
# =-=-=-=-=-=-=-=-=-=-=
# LOAD the CSV into a dataframe
# =-=-=-=-=-=-=-=-=-=-= 

import pandas

# Let python create the column names list:
with open('./TEDtalks_2018_edited.csv') as f:
    colnames = f.readline().strip().split(",")

df = pandas.read_csv('./TEDtalks_2018_edited.csv', names=colnames)

df.head(10)

,Talk ID,public_url,speaker_name,headline,description_x,event,duration_x,language,published,tags,speaker,duration_y,uploaded,views,description_y,text
0,Talk ID,public_url,speaker_name,headline,description_x,event,duration_x,language,published,tags,speaker,duration_y,uploaded,views,description_y,text
1,1,https://www.ted.com/talks/al_gore_on_averting_...,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,en,6/27/06,"alternative energy,cars,global issues,climate ...",Al Gore,PT16M17S,2006-06-27T00:11:00+00:00,3266733,With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a g..."
2,7,https://www.ted.com/talks/david_pogue_says_sim...,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,en,6/27/06,"simplicity,entertainment,interface design,soft...",David Pogue,PT21M26S,2006-06-27T00:11:00+00:00,1702201,New York Times columnist David Pogue takes aim...,"(Music: ""The Sound of Silence,"" Simon & Garf..."
3,53,https://www.ted.com/talks/majora_carter_s_tale...,Majora Carter,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,en,6/27/06,"MacArthur grant,cities,green,activism,politics...",Majora Carter,PT18M36S,2006-06-27T00:11:00+00:00,2000421,"In an emotionally charged talk, MacArthur-winn...",If you're here today — and I'm very happy th...
4,66,https://www.ted.com/talks/ken_robinson_says_sc...,Ken Robinson,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,en,6/27/06,"children,teaching,creativity,parenting,culture...",Ken Robinson,PT19M24S,2006-06-27T00:11:00+00:00,51614087,Sir Ken Robinson makes an entertaining and pro...,Good morning. How are you? (Laughter) ...
5,92,https://www.ted.com/talks/hans_rosling_shows_t...,Hans Rosling,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,en,6/27/06,"demo,Asia,global issues,visualizations,global ...",Hans Rosling,PT19M50S,2006-06-27T20:38:00+00:00,12662135,You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to te..."
6,96,https://www.ted.com/talks/tony_robbins_asks_wh...,Tony Robbins,Why we do what we do,"Tony Robbins discusses the ""invisible forces"" ...",TED2006,0:21:45,en,6/27/06,"entertainment,goal-setting,potential,psycholog...",Tony Robbins,PT21M45S,2006-06-27T20:38:00+00:00,22368699,"Tony Robbins discusses the ""invisible forces"" ...",Thank you. I have to tell you I'm both chall...
7,49,https://www.ted.com/talks/joshua_prince_ramus_...,Joshua Prince-Ramus,Behind the design of Seattle's library,Architect Joshua Prince-Ramus takes the audien...,TED2006,0:19:58,en,7/10/06,"library,architecture,design,culture,collaboration",Joshua Prince-Ramus,PT19M58S,2006-07-10T00:11:00+00:00,1042335,Architect Joshua Prince-Ramus takes the audien...,I'm going to present three projects in rapid...
8,86,https://www.ted.com/talks/julia_sweeney_on_let...,Julia Sweeney,Letting go of God,When two young Mormon missionaries knock on Ju...,TED2006,0:16:32,en,7/10/06,"atheism,Christianity,religion,God,comedy,humor...",Julia Sweeney,PT16M32S,2006-07-10T00:11:00+00:00,3903747,When two young Mormon missionaries knock on Ju...,"On September 10, the morning of my seventh b..."
9,71,https://www.ted.com/talks/rick_warren_on_a_lif...,Rick Warren,A life of purpose,"Pastor Rick Warren, author of ""The Purpose-Dri...",TED2006,0:21:02,en,7/18/06,"Christianity,philanthropy,religion,God,happine...",Rick Warren,PT21M2S,2006-07-18T00:11:00+00:00,3361934,"Pastor Rick Warren, author of ""The Purpose-Dri...","I'm often asked, ""What surprised you about t..."


Okay, I'm not sure why we are seeing `NaN` in the 6 rightmost columns above: speaker, duration, uploaded, views, description, text. They are displaying properly in the Jupyter Lab CSV viewer. Viewing them in the JL editor -- *hey! same initials as me!*, it looks like they were not wrapped in quotations marks making them strings. So, now the question is whether we edit the CSV file or simply re-generate the merged file. (I think the latter may be simpler.)

**SOLUTION**: `NaN` was appearing because I had copied and pasted code and had the wrong `CSV` in `pandas.read_csv()`.

### Comparing Columns

We know that some of the columns are duplicates. We need to assess each set of possible duplicates. First, a quick reminder of all the columns involved, then some exploration of the differences involved: it looks like, for example, the descriptions drawn from the Google document have had fancier punctuation -- e.g., smart quotes and em dashes -- replaced. This matters little, so differences here are trivial: but it would help to establish this by removing punctuation and comparing just the words. 

Some of the early work on comparing texts revisited old methods, including the NLTK's white space tokenizer, but none of them got rid of the en or em dash in my example. (So it was a good example for having that persistent problem.)

```python
    import string
    table = str.maketrans({key: key for key in string.punctuation})
    print(df['description_y'][6].translate(table))
    
    import re
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    print(regex.sub('', df['description_y'][6].translate(table)))

    from nltk.tokenize import WhitespaceTokenizer
    print(WhitespaceTokenizer().tokenize(df['description_y'][6]))
```
    
Regexing got tricky. Here's what I tried:

* `r'[^\w\s]'`: **failed** by removing the hyphen in "high-five".
* `r'[^-\w]+'`: **failed** by removing the hyphen in "high-five".
* `r'(?:(?!-)\W)+'`: **passed**

In [3]:
print(colnames)

['Talk ID', 'public_url', 'speaker_name', 'headline', 'description_x', 'event', 'duration_x', 'language', 'published', 'tags', 'speaker', 'duration_y', 'uploaded', 'views', 'description_y', 'text']


In [21]:
row = 4
print(df['description_x'][row] == df['description_y'][row])
print(df['description_x'][row] + "\n" + df['description_y'][row])

True
Sir Ken Robinson makes an entertaining and profoundly moving case for creating an education system that nurtures (rather than undermines) creativity.
Sir Ken Robinson makes an entertaining and profoundly moving case for creating an education system that nurtures (rather than undermines) creativity.


In [23]:
row = 6
print(df['description_x'][row] == df['description_y'][row])
print(df['description_x'][row] + "\n" + df['description_y'][row])

False
Tony Robbins discusses the "invisible forces" that motivate everyone's actions -- and high-fives Al Gore in the front row.
Tony Robbins discusses the "invisible forces" that motivate everyone's actions — and high-fives Al Gore in the front row.


In [56]:
import re

def keep_hyphens(text):
    return re.sub(r'(?:(?!-)\W)+', ' ', text)

In [53]:
print(re.sub(r'(?:(?!-)\W)+', ' ', df['description_y'][6]))

Tony Robbins discusses the invisible forces that motivate everyone s actions and high-fives Al Gore in the front row 


In [59]:
row = 6
print(keep_hyphens(df['description_x'][row]) == keep_hyphens(df['description_y'][row]))
print(keep_hyphens(df['description_x'][row]) + "\n" + keep_hyphens(df['description_y'][row]))

False
Tony Robbins discusses the invisible forces that motivate everyone s actions -- and high-fives Al Gore in the front row 
Tony Robbins discusses the invisible forces that motivate everyone s actions and high-fives Al Gore in the front row 


In [5]:
import numpy as np

different = np.where(df['description_x'] != df['description_y'])

high_scores1['is_score_chased'] = np.where(high_scores1['runs1']>=high_scores1['runs2'], 
                                           'yes', 'no')

In [10]:
print(different)

(array([   0,    2,    3, ..., 2683, 2684, 2686]),)


In [ ]:
for row in df:
    if column_a != column_b
    print(row_id)